In [13]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from pathlib import Path
from tqdm import tqdm_notebook, tqdm, trange
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.utils import to_categorical
from DataPrep import preprocess_image, augment_pipeline

In [2]:
train_df = pd.read_csv('datasets/train_sets.csv')
val_df = pd.read_csv('datasets/val_sets.csv')

In [3]:
train_df

,Image,Id,whale_id_index
0,66f164af.jpg,new_whale,0
1,81bd5469.jpg,w_143b201,20
2,193ae7a0.jpg,w_5297ab3,95
3,feb15de4.jpg,w_8d46cef,167
4,7fe190ce.jpg,w_43b50e5,76
...,...,...,...
2649,53c207cf.jpg,w_94cd45e,176
2650,8cc05900.jpg,w_7554f44,140
2651,fa8ed4af.jpg,w_43be268,77
2652,59ecd346.jpg,w_2a04ceb,43


In [14]:

class DataGenerator:
    def __init__(self, df, dataset_dir, batch_size=32):
        self.df = df
        self.dataset_dir = dataset_dir
        self.number_class = self.df['whale_id_index'].max() + 1
        self.batch_size = batch_size
    
    @property
    def number_step(self):
        return len(self.df) // self.batch_size
    
    def create_generator(self, shuffle=False, augment=False):
        df = self.df
        if shuffle:
            df = df.sample(frac=1, replace=False)
        while True:
            for i in range(0, len(df), self.batch_size):
                batch_df = df.iloc[i:i+self.batch_size]
                
                xs = []
                ys = []
                for _, whale in batch_df.iterrows():
                    img = cv2.imread(str(self.dataset_dir / whale['Image']))
                    if augment:
                        img = augment_pipeline.augment(image=img)
                    img = preprocess_image(img)
                    
                    _class = to_categorical(whale['whale_id_index'], self.number_class)
                    
                    xs.append(img)
                    ys.append(_class)
                    
                xs = np.array(xs)
                ys = np.array(ys)
                yield xs, ys
                

In [15]:
input_dir = Path('datasets/train/train')

train_datagen = DataGenerator(train_df, input_dir)
train_gen = train_datagen.create_generator(shuffle=True, augment=False)

val_datagen = DataGenerator(val_df, input_dir)
val_gen = val_datagen.create_generator(shuffle=False)

In [16]:
backbone = DenseNet121(include_top=False,
                       weights='imagenet',
                       input_shape=(478, 968, 3)
                      )
model = Sequential([
    backbone,
    GlobalAveragePooling2D(),
    Dense(train_datagen.number_class, activation='softmax')
])

In [7]:
backbone.trainable = False

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 15, 30, 1024)      7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 313)               320825    
Total params: 7,358,329
Trainable params: 320,825
Non-trainable params: 7,037,504
_________________________________________________________________


In [9]:
optimizer = Adam(1e-3)
model.compile(optimizer,
              loss='categorical_crossentropy',
              metrics=['acc'],
             )

In [12]:
model_dir = Path('benchmark/baseline')
model_dir.mkdir(exist_ok=True)

model.fit(train_gen,
          steps_per_epoch=train_datagen.number_step,
          validation_data=val_gen,
          validation_steps=val_datagen.number_step,
          epochs=75,
          callbacks=[
              TensorBoard(str(model_dir)),
              ModelCheckpoint(str(model_dir / 'freeze-ep-{epoch:02d}-val_acc{val_acc:.4f}.hdf5'),
                        monitor='val_acc',
                        verbose=1,
                        save_best_only=True,
                        mode='max')
          ]
          
)

Epoch 1/75
82/82 [==============================] - ETA: 0s - loss: 5.2144 - acc: 0.2269
Epoch 00001: val_acc improved from -inf to 0.24688, saving model to benchmark/baseline/freeze-ep-01-val_acc0.2469.hdf5
82/82 [==============================] - 55s 668ms/step - loss: 5.2144 - acc: 0.2269 - val_loss: 4.7075 - val_acc: 0.2469
Epoch 2/75
82/82 [==============================] - ETA: 0s - loss: 4.6283 - acc: 0.2450
Epoch 00002: val_acc did not improve from 0.24688
82/82 [==============================] - 49s 603ms/step - loss: 4.6283 - acc: 0.2450 - val_loss: 4.5046 - val_acc: 0.2469
Epoch 3/75
82/82 [==============================] - ETA: 0s - loss: 4.3387 - acc: 0.2452
Epoch 00003: val_acc did not improve from 0.24688
82/82 [==============================] - 49s 601ms/step - loss: 4.3387 - acc: 0.2452 - val_loss: 4.3367 - val_acc: 0.2469
Epoch 4/75
82/82 [==============================] - ETA: 0s - loss: 4.0773 - acc: 0.2468
Epoch 00004: val_acc did not improve from 0.24688
82/82 [==

Epoch 27/75
82/82 [==============================] - ETA: 0s - loss: 1.3656 - acc: 0.7086
Epoch 00027: val_acc improved from 0.39375 to 0.40625, saving model to benchmark/baseline/freeze-ep-27-val_acc0.4062.hdf5
82/82 [==============================] - 49s 598ms/step - loss: 1.3656 - acc: 0.7086 - val_loss: 2.7969 - val_acc: 0.4062
Epoch 28/75
82/82 [==============================] - ETA: 0s - loss: 1.3123 - acc: 0.7223
Epoch 00028: val_acc improved from 0.40625 to 0.41563, saving model to benchmark/baseline/freeze-ep-28-val_acc0.4156.hdf5
82/82 [==============================] - 49s 594ms/step - loss: 1.3123 - acc: 0.7223 - val_loss: 2.7717 - val_acc: 0.4156
Epoch 29/75
82/82 [==============================] - ETA: 0s - loss: 1.2616 - acc: 0.7338
Epoch 00029: val_acc improved from 0.41563 to 0.41875, saving model to benchmark/baseline/freeze-ep-29-val_acc0.4187.hdf5
82/82 [==============================] - 49s 598ms/step - loss: 1.2616 - acc: 0.7338 - val_loss: 2.7477 - val_acc: 0.418

82/82 [==============================] - 50s 612ms/step - loss: 0.5662 - acc: 0.9241 - val_loss: 2.4523 - val_acc: 0.4875
Epoch 53/75
82/82 [==============================] - ETA: 0s - loss: 0.5489 - acc: 0.9302
Epoch 00053: val_acc improved from 0.48750 to 0.48906, saving model to benchmark/baseline/freeze-ep-53-val_acc0.4891.hdf5
82/82 [==============================] - 50s 606ms/step - loss: 0.5489 - acc: 0.9302 - val_loss: 2.4445 - val_acc: 0.4891
Epoch 54/75
82/82 [==============================] - ETA: 0s - loss: 0.5313 - acc: 0.9317
Epoch 00054: val_acc improved from 0.48906 to 0.49219, saving model to benchmark/baseline/freeze-ep-54-val_acc0.4922.hdf5
82/82 [==============================] - 50s 612ms/step - loss: 0.5313 - acc: 0.9317 - val_loss: 2.4380 - val_acc: 0.4922
Epoch 55/75
82/82 [==============================] - ETA: 0s - loss: 0.5148 - acc: 0.9336
Epoch 00055: val_acc improved from 0.49219 to 0.49375, saving model to benchmark/baseline/freeze-ep-55-val_acc0.4938.hdf